In [1]:
import baltic as bt
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from datetime import datetime as dt
from datetime import timedelta
import time
#import pymc3
import math
import arviz as az
#from hpd import hpd
import scipy.stats as stats
from io import StringIO
import altair as alt
from altair import datum
alt.data_transformers.disable_max_rows()
import seaborn as sns

from zipfile import ZipFile
import scipy as sp


import sys, subprocess, glob, os, shutil, re, importlib
from subprocess import call
import imp


%matplotlib inline
import matplotlib as mpl
from matplotlib import pyplot as plt
import matplotlib.patheffects as path_effects
import matplotlib.lines as mlines
from matplotlib.font_manager import FontProperties
import matplotlib.colors as clr
from matplotlib import rc
import textwrap as textwrap
from textwrap import wrap

import warnings
warnings.filterwarnings('ignore')

from scipy.special import binom

In [2]:
##reads in all the Nes from BEAST log files
def read_in_Ne_changes_mascot(log_file_path):
    
    Ne_skyline_dict = {"sample":[]}
    
    with open(log_file_path, "r") as infile:
        line_number = 0
        for line in infile:
            line_number += 1
            if not line.startswith("#"):  # log combiner will sometimes put the entire xml at the start of the log file
                # use the first line to find the migration rate columns
                #print(line)
            # use the first line to find the migration rate columns
                if "posterior" in line:
                    all_cols = line.split("\t")
                    Ne_column_indices = []   # list to store column indices
                    Nes_key = {}   # dictionary to store the column index to map to column name

                    for i in range(len(all_cols)):
                        col = all_cols[i]
                        if "Ne." in col:
                            Ne_column_indices.append(i)

                    # make an empty dictionary to store Nes and generate dictionary to convert index to name
                    for n in Ne_column_indices:
                        name = line.split("\t")[n]
                        #deme = name.split(".")[1]# the syntax here is "Ne_region.1" where region is deme and 1 is interval 1
                        interval = name.split(".")[1]
                       
                        Nes_key[n] = name
                        Ne_skyline_dict[name] = []


                # read in actual parameter estimates and store in dictionary
                else:
                    sample = line.split("\t")[0]
                    Ne_skyline_dict["sample"].append(sample)

                    for index in Ne_column_indices:
                        name = Nes_key[index]
                        Ne_skyline_dict[name].append(line.split("\t")[index])
                    
                
    return(Ne_skyline_dict)


In [3]:
# make a new dataframe that summarizes the 95% HPD estimate with mean for each deme and interval 
def generate_summary_df(input_df):
    
    
    new_df = pd.DataFrame()

    for i in input_df.columns.tolist():
        if "Ne." in i:
            #deme = i.split(".")[1]
           # print(deme)
            #interval = 
            #print(interval)
#             if "\n" in i.split(".")[2]:
#                 interval = i.split(".")[2][0:2]
#             else:
            interval = i.split(".")[1]
           # print(interval)
            #print(interval)
            #print(i)
            #next_interval = int(interval)+1
            local_series = input_df[i].astype('float').to_numpy()
            #print(local_series)
            mean_log = local_series.mean()
            median_log = np.median(local_series)
            mean_linear = np.mean(np.exp(local_series))
            median_linear = np.median(np.exp(local_series))
            hpd_95 = az.hdi(local_series, 0.95)
            lower_hpd_log_95 = hpd_95[0]
            lower_hpd_linear_95 = math.exp(lower_hpd_log_95)
            upper_hpd_log_95 = hpd_95[1]
            upper_hpd_linear_95 = math.exp(upper_hpd_log_95)
            hpd_50 = az.hdi(local_series, 0.50)
            lower_hpd_log_50 = hpd_50[0]
            lower_hpd_linear_50 = math.exp(lower_hpd_log_50)
            upper_hpd_log_50 = hpd_50[1]
            upper_hpd_linear_50 = math.exp(upper_hpd_log_50)
            
#             try:
#                 next_local_series = input_df["SkylineNe"+"."+ str(deme) +"." + str(next_interval)].astype('float').to_numpy()
#                 diff_series = np.subtract(local_series, next_local_series)
#                 #print(local_series)
#                 #print(next_local_series)
#                 #print(diff_series)
#                 diff_mean_log = diff_series.mean()
#                 diff_median_log = np.median(diff_series)
#                 diff_hpd_95 = az.hdi(diff_series, 0.95)
#                 diff_lower_hpd_log_95 = diff_hpd_95[0]
#                 diff_lower_hpd_linear_95 = math.exp(diff_lower_hpd_log_95)
#                 diff_upper_hpd_log_95 = diff_hpd_95[1]
#                 diff_upper_hpd_linear_95 = math.exp(diff_upper_hpd_log_95)
#                 diff_hpd_50 = az.hdi(diff_series, 0.50)
#                 diff_lower_hpd_log_50 = diff_hpd_50[0]
#                 diff_lower_hpd_linear_50 = math.exp(diff_lower_hpd_log_50)
#                 diff_upper_hpd_log_50 = diff_hpd_50[1]
#                 diff_upper_hpd_linear_50 = math.exp(diff_upper_hpd_log_50)
#             except KeyError:
#                 pass   
            
            try:
                local_df = pd.DataFrame.from_dict({"interval":interval, "mean_Ne_log":mean_log,"mean_Ne_linear":mean_linear, 
                                                   "median_Ne_log" : median_log, "median_Ne_linear": median_linear, 
                                                   "upper_hpd_log_95":upper_hpd_log_95,"lower_hpd_log_95":[lower_hpd_log_95], 
                                                   "upper_hpd_log_50":upper_hpd_log_50,"lower_hpd_log_50":lower_hpd_log_50,
                                                   "upper_hpd_linear":upper_hpd_linear_95,"lower_hpd_linear":lower_hpd_linear_95,
                                                   "upper_hpd_linear_50":upper_hpd_linear_50, "lower_hpd_linear_50":lower_hpd_linear_50,
                                                  })
                new_df = new_df.append(local_df)
                #print(new_df)
            except:
                pass
            
    return(new_df)

In [4]:
#calculating transmission rate
def generate_summary_diff_df(input_df):
    
    
    new_df = pd.DataFrame()
   
    for i in input_df.columns.tolist():
        if "Ne." in i:
            #deme = i.split(".")[1]
           # print(deme)
            #interval = 
            #print(interval)
#             if "\n" in i.split(".")[2]:
#                 interval = i.split(".")[2][0:2]
#             else:
            interval = i.split(".")[1]
            next_interval = int(interval)+3 #averaging over two weeks to reduce noise
            local_series = input_df[i].astype('float').to_numpy()
            #print(local_series)
           
            try:
                new_df["Ne"+ ".diff." + str(interval)] = (52/5)*(input_df[i].astype("float") - input_df["Ne"+"." + str(next_interval)].astype('float'))
            
            
            except KeyError:
                pass 
            
            
    return(new_df)

In [5]:
#calculates Rt with both intros and local transmission 
def generate_local_and_intro_rt(input_df):
    
    
    new_df = pd.DataFrame()
    incubation_period = 365/14
    uninfectious_rate = 365/4.5
    
    for i in input_df.columns.tolist():
        if "Ne" in i:
            interval = i.split(".")[2]
            new_df["rt" +  "." + str(interval)] = (1+ (input_df[i].astype("float") / uninfectious_rate)) * (1+ (input_df[i].astype("float") / incubation_period))
            

            
    return(new_df)

In [6]:
# make a new dataframe that summarizes the 95% HPD estimate with mean for each deme and interval 
def generate_rt_summary_df(input_df):
    
    
    new_df = pd.DataFrame()
    count = 0
    for i in input_df.columns.tolist():
        #print(i)
        if "rt" in i:
            if count %1 == 0:
                interval = i.split(".")[1]
                #interval = i.split(".")[2]
                local_series = input_df[i].astype('float').to_numpy()
                mean_percent = local_series.mean()
                median_percent = np.median(local_series)

                hpd_95 = az.hdi(local_series, 0.95)
                lower_hpd_log_95 = hpd_95[0]
                upper_hpd_log_95 = hpd_95[1]
                hpd_50 = az.hdi(local_series, 0.50)
                lower_hpd_log_50 = hpd_50[0]
                upper_hpd_log_50 = hpd_50[1]

                try:
                    local_df = pd.DataFrame.from_dict({"interval":interval, "mean_percent":mean_percent, "median_percent":median_percent,
                                                       "upper_hpd_log_95":upper_hpd_log_95,"lower_hpd_log_95":[lower_hpd_log_95], 
                                                       "upper_hpd_log_50":upper_hpd_log_50,"lower_hpd_log_50":lower_hpd_log_50})
                    new_df = new_df.append(local_df)
                except:
                    pass
            count+=1

    return(new_df)

In [7]:
# read forward migration rates from BEAST log files
def read_in_forward_migration_rates_mascot(log_file_path):
    
    mig_rates_dict = {"sample":[]}
    
    with open(log_file_path, "r") as infile:
        line_number = 0
        for line in infile:
            #print(line_number)
            line_number += 1
            if not line.startswith("#"):  # log combiner will sometimes put the entire xml at the start of the log file
                # use the first line to find the migration rate columns
                
            # use the first line to find the migration rate columns
                if "posterior" in line:
                    all_cols = line.split("\t")
                    mig_column_indices = []   # list to store column indices
                    mig_key = {}   # dictionary to store the column index to map to column name

                    for i in range(len(all_cols)):
                        col = all_cols[i]
                        if "immigrationRate" in col: 
                            mig_column_indices.append(i)

                    # make an empty dictionary to store Nes and generate dictionary to convert index to name
                    for n in mig_column_indices:
                        name = line.split("\t")[n]
                        interval = name.split(".")[1]# the syntax here is "NeLog.state01" where 0 is deme and 1 is interval 1
                        #interval = name.split(".")[2]
                       
                        mig_key[n] = name
                        mig_rates_dict[name] = []


                # read in actual parameter estimates and store in dictionary
                else:
                    sample = line.split("\t")[0]
                    mig_rates_dict["sample"].append(sample)

                    for index in mig_column_indices:
                        name = mig_key[index]
                        mig_rates_dict[name].append(line.split("\t")[index])
                    
                
                
                
    return(mig_rates_dict)

In [8]:
# calculates the percentage of cases due to introductions. Need both the migration df and the estimates transmisssion rate from the differences in log Nes
def generate_percent_intro_df(input_df, seir_transmission_rate):
    
    temp_df = pd.DataFrame()
    new_df = pd.DataFrame()
   
    for i in input_df.columns.tolist():
        if "immigrationRate" in i:
        
            interval = i.split(".")[1]
            #deme = i.split(".")[2]

            try:
                temp_df["total."+ str(interval)] = seir_transmission_rate["Ne.diff." + str(interval)].astype("float") +  input_df[i].astype("float").apply(np.exp)
                new_df["intro.percent"+ "." + str(interval)] = input_df[i].astype("float").apply(np.exp).div(temp_df["total."+ str(interval)], axis = 0) 

            except KeyError: #this was added because not all regions have equal time periods for their epidemics and it was throwing an error everytime it had to switch deme
                pass 
                  
    return(new_df)

In [9]:
# make a new dataframe that summarizes the 95% HPD estimate with mean for each deme and interval 
def generate_summary_df_mig(input_df):
    
    
    new_df = pd.DataFrame()

    for i in input_df.columns.tolist():
        if "percent" in i:
            interval = i.split(".")[2]
            #interval = i.split(".")[3]
            local_series = input_df[i].astype('float').to_numpy()
            #print(local_series)
            mean_percent = np.median(local_series)
            hpd_95 = az.hdi(local_series, 0.95)
            lower_hpd_log_95 = hpd_95[0]
            upper_hpd_log_95 = hpd_95[1]
            hpd_50 = az.hdi(local_series, 0.50)
            lower_hpd_log_50 = hpd_50[0]
            upper_hpd_log_50 = hpd_50[1]
            

            
            
            try:
                local_df = pd.DataFrame.from_dict({ "interval":interval, "mean_percent":mean_percent, 
                                                   "upper_hpd_log_95":upper_hpd_log_95,"lower_hpd_log_95":[lower_hpd_log_95], 
                                                   "upper_hpd_log_50":upper_hpd_log_50,"lower_hpd_log_50":lower_hpd_log_50})
                new_df = new_df.append(local_df)
            except:
                pass
            
    return(new_df)


In [14]:
# now do the actual analyses
def plot_ne_and_rt_comparison(log_file_path, simulation_path):
    # read in log file for changes in Ne
    Ne_skyline = read_in_Ne_changes_mascot(log_file_path)

    # remove burn-in
    Ne_df = pd.DataFrame.from_dict(Ne_skyline)

    burnin_percent = 0.2
    print(len(Ne_df))
    rows_to_remove = int(len(Ne_df)* burnin_percent)
    Ne_df = Ne_df.iloc[rows_to_remove:]

    print(len(Ne_df))
    Ne_df = Ne_df.reset_index()
    
#     ne_summary = generate_summary_df(Ne_df)

#     # format time intervals
#     test = ne_summary
#     test['days'] = (test.interval.astype(int))*7
#     test['date'] = dt.strptime("2023-01-14",  "%Y-%m-%d") - test.days.map(timedelta)
#     test["new_interval"] = (((test.interval.iloc[::-1]).astype(int))*7 )-7

#     # add smoothing 
#     columns = ["mean_Ne_log","mean_Ne_linear",  "median_Ne_linear","median_Ne_log", "upper_hpd_log_50","lower_hpd_log_50", "upper_hpd_linear_50","lower_hpd_linear_50", "lower_hpd_linear", "upper_hpd_linear"]
#     for column in columns:
#         # Create a new column for the moving average
#         test[f'{column}_MA'] = test[column].rolling(4, min_periods =1).mean()


#     # plot the Ne over time for sims analyzed via multi-tree coal
#     range_ = ['#EEC060', '#2664A5', '#A76BB1', "#EEA160", "#356D4C"]

#     log_line_ne = alt.Chart(test).mark_area(interpolate='monotone', opacity = 0.8, clip = True).encode(
#         alt.X('new_interval', ),
#         alt.Y('upper_hpd_log_50_MA',axis=alt.Axis(title="", grid=False), scale = alt.Scale(domain=[-3,8])),
#         alt.Y2('lower_hpd_log_50_MA' ),    
#     ).properties(
#         width=850,
#         height=300
#     )

#     log_band_ne = alt.Chart(test).mark_line(interpolate='monotone', opacity = 0.5, clip = True, color = "black").encode(
#         alt.X('new_interval', ),
#         alt.Y('median_Ne_log_MA',axis=alt.Axis(title="Effective Population Size", grid=False)),#,scale=alt.Scale(domain=(0, 13))),   
#     ).properties(
#         width=850,
#         height=300
#     )



    ## calculate SEIR transmission rate using the uninfectious period and the incubation period. 
    uninfectious_rate = 365/4.5 #taken from https://www.medrxiv.org/content/10.1101/2022.08.17.22278897v1.full.pdf
    incubation_period = 365/8

    ne_diff_summary = generate_summary_diff_df(Ne_df) #summarize data in easier datatable format
    seir_transmission_rate = ((ne_diff_summary*2 + uninfectious_rate + incubation_period)**2 - (incubation_period- uninfectious_rate)**2)/(4*incubation_period)


    ## read in simulation data directly from MASTER json. 
    simulation_df = pd.read_json(simulation_path, orient = "index")
    simulation_df = simulation_df.T

    ## log transform for easier comparison with BEAST output
    simulation_df.I[simulation_df.I < 0.1] = 0.1
    simulation_df.I = simulation_df.I.astype("float")
    simulation_df["log_I"] = simulation_df.I.apply(np.log)
    simulation_df["new_time"] = simulation_df.t *366


#     #plot the log cases from simulations
#     line_simulation = alt.Chart(simulation_df).mark_line(interpolate='monotone', opacity = 0.8, clip = True, color = "orange").encode(
#         alt.X('new_time', axis=alt.Axis(title="time interval")),
#         alt.Y('log_I',axis=alt.Axis(title="Simulated prevalence", grid=False)),    
#     ).properties(
#         width=850,
#         height=300)


#     final_ne_vs_sim_plot = (line_simulation + log_line_ne + log_band_ne).resolve_scale(x = "shared", y = "shared")

    ####### calculate Rt from BEAST 

    rt_local_and_intro_df = generate_local_and_intro_rt(ne_diff_summary)
    summary_rt_local_and_intro_df = generate_rt_summary_df(rt_local_and_intro_df)
    summary_rt_local_and_intro_df["new_interval"] = (((summary_rt_local_and_intro_df.interval.iloc[::-1]).astype(int))+.5)    
    columns = ["mean_percent","median_percent", "upper_hpd_log_95","lower_hpd_log_95", "upper_hpd_log_50", "lower_hpd_log_50"]
    for column in columns:
        # Create a new column for the moving average
        summary_rt_local_and_intro_df[f'{column}_MA'] = summary_rt_local_and_intro_df[column].rolling(4, min_periods =1, center = True).mean()


    ## plot BEAST Rt
    band_rt = alt.Chart(summary_rt_local_and_intro_df).mark_area(interpolate='monotone', opacity = 0.5, clip = True, color = "orange").encode(
        alt.X('new_interval', axis=alt.Axis(title="time interval")),
        alt.Y('upper_hpd_log_50_MA',axis=alt.Axis(title="Rt", grid=False)),
        alt.Y2("lower_hpd_log_50_MA")  
    ).properties(
        width=550,
        height=200)

    line_rt = alt.Chart(summary_rt_local_and_intro_df).mark_line(interpolate='monotone', opacity = 0.5, clip = True, color = "black").encode(
        alt.X('new_interval', axis=alt.Axis(title="time interval")),
        alt.Y('median_percent_MA',axis=alt.Axis(title="Rt", grid=False)),
    ).properties(
        width=550,
        height=200)

    ###### calculate Rt from simulations
    simulation_df_copy = simulation_df.copy()

    simulation_df_copy["whole_time"] = np.ceil(simulation_df_copy.t *366)

    daily_sim_cases = simulation_df_copy.groupby(["whole_time"]).agg({'IntroCount':'mean','I':'mean'}).reset_index()


    daily_sim_cases["diff_intro"] = daily_sim_cases.IntroCount.diff()
    daily_sim_cases["diff_I"] = daily_sim_cases.I.diff() 

    daily_sim_cases["log_I"] = daily_sim_cases.I.apply(np.log)
    daily_sim_cases["diff"] = daily_sim_cases.I.diff()

    #daily_sim_cases["log_diff"] = daily_sim_cases["diff"].apply(np.log) *365/7

    #daily_sim_cases["diff_I"][daily_sim_cases["diff_I"] < 0 ] = 0
    #daily_sim_cases["diff_intro"][daily_sim_cases["diff_intro"] < 0 ] = 0

    #define to week number
    counter = 0
    daily_sim_cases["week_time"] = np.nan
    for index, row in daily_sim_cases.iterrows():
        if row.whole_time %7 == 0:
            daily_sim_cases.loc[index,"week_time"] = counter 
            counter +=1

    #backfill
    daily_sim_cases = daily_sim_cases.bfill()

    # aggregate based on weeks
    weekly_sim_cases = daily_sim_cases.groupby(["week_time"]).agg({'diff_intro':'mean','I':'mean'}).reset_index()

    weekly_sim_cases["log_I"] = weekly_sim_cases.I.apply(np.log)
    weekly_sim_cases["diff"] = weekly_sim_cases.log_I.diff() *(365/7)


    incubation_period = 365/8
    uninfectious_rate = 365/4.5

    weekly_sim_cases["Rt"] = (1+ (weekly_sim_cases["diff"] / uninfectious_rate)) * (1+ (weekly_sim_cases["diff"] / incubation_period))
    weekly_sim_cases.week_time = (weekly_sim_cases.week_time -1.5) #*7
    weekly_sim_cases = weekly_sim_cases.iloc[3:]
    

    columns = ["Rt"]
    for column in columns:
        # Create a new column for the moving average
        weekly_sim_cases[f'{column}_MA'] = weekly_sim_cases[column].rolling(4, min_periods =1).mean()

    summary_rt_local_and_intro_df = summary_rt_local_and_intro_df.sort_values(by = ["new_interval"])
    combinded_ne_and_sims = pd.concat([weekly_sim_cases, summary_rt_local_and_intro_df.set_index(weekly_sim_cases.index)], axis=1)


    #plot Rt from simulations
    line_rt_sim = alt.Chart(weekly_sim_cases).mark_line(interpolate='monotone', opacity = 0.8, clip = True, color = "blue").encode(
        alt.X('week_time', axis=alt.Axis(title="")),
        alt.Y('Rt_MA',axis=alt.Axis(title="Rt", grid=False), scale = alt.Scale(domain = [0.3, 3])),   
    ).properties(
        width=550,
        height=200)

    final_rt_vs_sim_plot = (band_rt + line_rt+ line_rt_sim).resolve_scale(y ="shared")
    
    
    chart = alt.Chart(combinded_ne_and_sims).mark_circle().encode(
            x = alt.X('Rt_MA', axis=alt.Axis(title="Simulation Rt")),
            y = alt.Y('median_percent', axis=alt.Axis(title="Estimated Rt"))
    )

    final_rt_corr = chart + chart.transform_regression('Rt_MA','median_percent').mark_line() + chart.transform_regression('Rt_MA','median_percent', groupby=["sampling"], params = True).mark_text().encode(
        x=alt.value(150),  # pixels from left
        y=alt.value(50),  # pixels from top
        text='params:N',
    ).transform_calculate(
        params='"rSquared = " + round((datum.rSquared) * 100)/100 '
    ).properties(
        width=550,
        height=200)


    return final_rt_vs_sim_plot, final_rt_corr

In [15]:
simulation_path = "../simulations/simulation_results/eir_3.json"
#log_file_path = "../simulations/beast_results/simmulticoal_4_all.log"

In [16]:
import os

folder_path = '../simulations/beast_results/simulation_3/'

# List all files in sims folder
log_files = [(folder_path + file) for file in os.listdir(folder_path) if file.endswith('.log')]

#put everything in order
log_files.sort()

In [17]:
chart_list_ne = []
chart_list_rt = []

for file in log_files:
    base_filename = file.split("/")[3].split(".")[0]
    
    plot1, plot2 = plot_ne_and_rt_comparison(file, simulation_path)
    plot1 = plot1.properties(title="Rt values - estimated vs simulation:" + file.split("_")[-2] + "_"+ file.split("_")[-1].split(".")[0])
    plot2 = plot2.properties(title="Rt correlation- estimated vs simulation:" + file.split("_")[-2] + "_"+ file.split("_")[-1].split(".")[0])

    
    chart_list_ne.append(plot1)
    chart_list_rt.append(plot2)

5001
4001
5001
4001
28184
22548
48667
38934
18196
14557
31787
25430


In [18]:
plots_1 = alt.vconcat(*chart_list_ne)

In [19]:
plots_2 = alt.vconcat(*chart_list_rt)

In [20]:
plots_1 | plots_2

alt.HConcatChart(...)

In [21]:
def plot_percent_intro_comparison(log_file_path, simulation_path):

    ### now working on percentage of cases due to introductions
    Ne_skyline = read_in_Ne_changes_mascot(log_file_path)

    # remove burn-in
    Ne_df = pd.DataFrame.from_dict(Ne_skyline)

    burnin_percent = 0.3
    print(len(Ne_df))
    rows_to_remove = int(len(Ne_df)* burnin_percent)
    Ne_df = Ne_df.iloc[rows_to_remove:]

    print(len(Ne_df))
    Ne_df = Ne_df.reset_index()
    

    ## calculate SEIR transmission rate using the uninfectious period and the incubation period. 
    uninfectious_rate = 365/4.5 #taken from https://www.medrxiv.org/content/10.1101/2022.08.17.22278897v1.full.pdf
    incubation_period = 365/8

    ne_diff_summary = generate_summary_diff_df(Ne_df) #summarize data in easier datatable format
    seir_transmission_rate = ((ne_diff_summary*2 + uninfectious_rate + incubation_period)**2 - (incubation_period- uninfectious_rate)**2)/(4*incubation_period)

    # read in migration rate estimates from bEAST
    migration_rates_f = read_in_forward_migration_rates_mascot(log_file_path)
    mig_df_f = pd.DataFrame.from_dict(migration_rates_f)

    # remove burnin
    burnin_percent = 0.3
    #print(len(mig_df_f))
    rows_to_remove = int(len(mig_df_f)* burnin_percent)
    mig_df_f = mig_df_f.iloc[rows_to_remove:]

    #print(len(mig_df_f))
    mig_df_f = mig_df_f.reset_index()

    # calculate % of cases due to introd
    percent_df = generate_percent_intro_df(mig_df_f, seir_transmission_rate)

    # summarize data in easier to use format
    summary_percent_df = generate_summary_df_mig(percent_df)
    summary_percent_df = summary_percent_df.reset_index()

    # making sure that any numbers >1 are excluded as outliers from stochastic noise
    percent_df =pd.DataFrame(np.where(percent_df <1, percent_df, 1), columns=percent_df.columns )
    percent_df =pd.DataFrame(np.where(percent_df >0, percent_df, 0), columns=percent_df.columns )

    # format dates
    summary_percent_df["new_interval"] = (((summary_percent_df.interval.iloc[::-1]).reset_index(drop=True).astype(int))+.5)   #smoothing
    summary_percent_df['days'] = (summary_percent_df.interval.astype(int)-3)*7 #the 1.5 adjustment is made due to the fact that we take the difference of Nes over three weeks 
    summary_percent_df['date'] = dt.strptime("2024-09-12",  "%Y-%m-%d") - summary_percent_df.days.map(timedelta)
    #print(summary_percent_df)
    # smoothing due to stochasitc nosie
    columns = ["mean_percent", "upper_hpd_log_95","lower_hpd_log_95", "upper_hpd_log_50", "lower_hpd_log_50"]
    for column in columns:
        # Create a new column for the moving average
        summary_percent_df[f'{column}_MA'] = summary_percent_df[column].rolling(3, min_periods =1).mean()

    #print(summary_percent_df)
    # plot % of cases due to intros from BEAST
    line1 = alt.Chart(summary_percent_df).mark_area(interpolate='monotone', opacity = 0.9, clip = True).encode(
        alt.X('new_interval', title = "Time Interval"),
        alt.Y('lower_hpd_log_50_MA',title = "% of cases due to intros", axis=alt.Axis( grid=False, format='%', ), scale=alt.Scale(domain=(0.0, 1))),
        alt.Y2('upper_hpd_log_50_MA' )
    ).properties(
        width=450,
        height=150)

    band = alt.Chart(summary_percent_df).mark_area(interpolate='monotone', opacity = 0.2, clip = True).encode(
        alt.X('new_interval', title = "Time Interval"),
        alt.Y('lower_hpd_log_95_MA',title = "% of cases due to intros", axis=alt.Axis( grid=False, format='%', ), scale=alt.Scale(domain=(0.0, 1))),
        alt.Y2('upper_hpd_log_95_MA' )
    ).properties(
        width=450,
        height=150)

    #line1
    intro_plot = (line1 + band).resolve_scale(y= "shared") 
    
    ## read in simulation dynamics
    simulation_df = pd.read_json(simulation_path, orient = "index")
    simulation_df = simulation_df.T

    ## log transform for easier comparison with BEAST output
    simulation_df.I[simulation_df.I < 0.1] = 0.1
    simulation_df.I = simulation_df.I.astype("float")
    simulation_df["log_I"] = simulation_df.I.apply(np.log)
    simulation_df["new_time"] = simulation_df.t *366

    ## now calculate the true percentage of cases due to intros from simulations
    simulation_df["whole_time"] = np.ceil(simulation_df.t *366)

    # aggregate to whole time due to the simulations having multiple events in the same day
    daily_sim_cases = simulation_df.groupby(["whole_time"]).agg({'IntroCount':'mean','I':'mean'}).reset_index()

    # take the difference to calcualte the incidence and new intros since I refers to the prevalence and not new cases
    daily_sim_cases["diff_intro"] = daily_sim_cases.IntroCount.diff()
    daily_sim_cases["diff_I"] = daily_sim_cases.I.diff()


    daily_sim_cases["diff_I"][daily_sim_cases["diff_I"] < 0 ] = 0
    daily_sim_cases["diff_intro"][daily_sim_cases["diff_intro"] < 0 ] = 0

    # changing daily to week
    counter = 0
    daily_sim_cases["week_time"] = np.nan
    for index, row in daily_sim_cases.iterrows():
        if row.whole_time %7 == 0:
            daily_sim_cases.loc[index,"week_time"] = counter 
            counter +=1
    daily_sim_cases = daily_sim_cases.bfill()
    weekly_sim_cases = daily_sim_cases.groupby(["week_time"]).agg({'diff_intro':'mean','I':'mean'}).reset_index()

    # calculate % of cases due to intros from sims
    weekly_sim_cases["percent_intro"] = weekly_sim_cases.diff_intro/(weekly_sim_cases.diff_intro+ weekly_sim_cases.I)

    weekly_sim_cases.week_time = (weekly_sim_cases.week_time -1.5) #*7
    weekly_sim_cases = weekly_sim_cases.iloc[3:]
    
    #smoothing - rolling mean 
    columns = ["percent_intro"]
    for column in columns:
        # Create a new column for the moving average
        weekly_sim_cases[f'{column}_MA'] = weekly_sim_cases[column].rolling(3, min_periods =1, center = True).mean()

    
    #plot from sims
    percent_intro_sim = alt.Chart(weekly_sim_cases).mark_line(interpolate='monotone', opacity = 0.8, clip = True, color = "black").encode(
        alt.X('week_time', axis=alt.Axis(title="Time Interval")),
        alt.Y('percent_intro_MA',axis=alt.Axis(title="% of cases due to intros", grid=False)),
    ).properties(
        width=450,
        height=150)

    final_plot = line1 +percent_intro_sim
    
        
    summary_percent_df = summary_percent_df.sort_values(by = ["new_interval"])

    combinded_percent_and_sims = pd.concat([weekly_sim_cases, summary_percent_df.set_index(weekly_sim_cases.index)], axis=1)

    
    chart = alt.Chart(combinded_percent_and_sims).mark_circle().encode(
        alt.X('percent_intro_MA', title = "Truth via Simulations"),
        alt.Y('mean_percent_MA', title = "Estimated via BEAST ", scale=alt.Scale(domain=(0.0, 0.4))
    ))

    final = chart + chart.transform_regression('percent_intro_MA','mean_percent_MA').mark_line() + chart.transform_regression('percent_intro_MA','mean_percent_MA', groupby=["sampling"], params = True).mark_text().encode(
        x=alt.value(150),  # pixels from left
        y=alt.value(50),  # pixels from top
        text='params:N',
    ).transform_calculate(
        params='"rSquared = " + round((datum.rSquared) * 100)/100 ').properties(
        width=450,
        height=150)

    
    return final_plot, final

In [26]:
chart_list_percent_intro = []
corr_list_plot = []

for file in log_files:
    # Get the base filename without the extension
    base_filename = file.split("/")[3].split(".")[0]
    
    # Assign the output of plot_ne_and_rt_comparison to variables
    plot1, plot2 = plot_percent_intro_comparison(file,simulation_path )
    
    plot1 = plot1.properties(title="Estimated vs simulation:" +file.split("_")[-2] + "_"+ file.split("_")[-1].split(".")[0])
    plot2 = plot2.properties(title="Correlation - estimated vs simulation:" +file.split("_")[-2] + "_"+ file.split("_")[-1].split(".")[0])


  
    chart_list_percent_intro.append(plot1)
    corr_list_plot.append(plot2)


5001
3501
5001
3501
28184
19729
48667
34067
18196
12738
31787
22251


In [27]:
percent_sim_plots = alt.vconcat(*chart_list_percent_intro)

In [28]:
percent_sims_corr = alt.vconcat(*corr_list_plot)

In [29]:
percent_sim_plots | percent_sims_corr

alt.HConcatChart(...)